## Setup

Before opening this notebook make sure you have run:

`setup lsst_sims -t sims`

from the command line to get the sims packages loaded.

Next we need to download some things.

* First is the small 3 square degree star database cached from Fatboy.

In [ ]:
!wget https://dirac.astro.washington.edu/~brycek/star_cache.db

   * Then there are two sets of CatSim support lightcurve template sets. One for the Kepler light curves and one for the mdwarf flares.

In [ ]:
! bash /home/docmaf/stack/stack/miniconda3-4.3.21-10a4fa6/Linux64/sims_catUtils/2.8.0.sims/support_scripts/get_kepler_light_curves.sh

In [ ]:
! bash /home/docmaf/stack/stack/miniconda3-4.3.21-10a4fa6/Linux64/sims_catUtils/2.8.0.sims/support_scripts/get_mdwarf_flares.sh

## Load modules and define classes

First we will load up the sims modules we will need.

In [ ]:
import numpy as np
import os
import pandas as pd
from lsst.utils import getPackageDir
from lsst.sims.catalogs.db import CatalogDBObject
from lsst.sims.utils import ObservationMetaData
from lsst.sims.catUtils.utils import ObservationMetaDataGenerator
from lsst.sims.catUtils.mixins import VariabilityStars, PhotometryStars, ParametrizedLightCurveMixin
from lsst.sims.catUtils.exampleCatalogDefinitions import PhoSimCatalogPoint
from lsst.sims.catalogs.decorators import cached, compound
from matplotlib import pyplot as plt
%matplotlib inline

The class below defines a `CatalogDBObject`. These are the classes that define how to connect to a database and retrieve the relevant columns for the type of object for which you want to create simulated catalogs. There are already defined functions for Stars that connect to the UW Fatboy database, but we need to create a new one to connect to the cached database we just downloaded.

The full database schema for Fatboy is available [here](https://confluence.lsstcorp.org/display/SIM/Database+Schema).

In [ ]:
class StarCacheDBObj(CatalogDBObject):
    tableid = 'star_cache_table'
    host = None
    port = None
    driver = 'sqlite'
    objectTypeId = 4
    idColKey = 'simobjid'
    raColName = 'ra'
    decColName = 'decl'

    columns = [('id','simobjid', int),
               ('raJ2000', 'ra*PI()/180.'),
               ('decJ2000', 'decl*PI()/180.'),
               ('glon', 'gal_l*PI()/180.'),
               ('glat', 'gal_b*PI()/180.'),
               ('properMotionRa', '(mura/(1000.*3600.))*PI()/180.'),
               ('properMotionDec', '(mudecl/(1000.*3600.))*PI()/180.'),
               ('parallax', 'parallax*PI()/648000000.'),
               ('galacticAv', 'CAST(ebv*3.1 as float)'),
               ('radialVelocity', 'vrad'),
               ('variabilityParameters', 'varParamStr', str, 256),
               ('sedFilename', 'sedfilename', str, 40)]

The second type of class we need to define is an Instance Catalog class. This will take the database queries and turn them into simulated catalogs for a given pointing and with the desired effects. There are basic catalog classes ready to go in the CatSim `sims_catUtils` repository. For example, there is `PhoSimCatalogPoint` which will define a basic instance catalog for point sources that's ready to be used as input to PhoSim for image simulations. Below we will take the basic `PhoSimCatalogPoint` class and add **mixins**. Mixins are python modules that are not meant to be in a class of their own and used in a wide range of other classes. CatSim has mixins for Photometry and Variability that we will use below.

In [ ]:
class testCatalogPoint(PhoSimCatalogPoint, VariabilityStars, PhotometryStars, ParametrizedLightCurveMixin):

    catalog_type = 'test_catalog_POINT'

    column_outputs = ['prefix', 'uniqueId', 'raPhoSim', 'decPhoSim', 'phoSimMagNorm', 'lsst_r', 'delta_lsst_r',
                      'sedFilepath',
                      'redshift', 'gamma1', 'gamma2', 'kappa', 'raOffset', 'decOffset',
                      'spatialmodel', 'internalExtinctionModel',
                      'galacticExtinctionModel', 'galacticAv', 'galacticRv', 'varParamStr']

    default_columns = [('redshift', 0., float), ('gamma1', 0., float), ('gamma2', 0., float),
                       ('kappa', 0., float), ('raOffset', 0., float), ('decOffset', 0., float),
                       ('galacticExtinctionModel', 'CCM', (str, 3)), ('galacticRv', 3.1, float),
                       ('internalExtinctionModel', 'none', (str, 4))]#, ('galacticAv', 0.04, float)]

    default_formats = {'S': '%s', 'f': '%.9g', 'i': '%i'}

    spatialModel = "point"

    transformations = {'raPhoSim': np.degrees, 'decPhoSim': np.degrees}
    
    disable_proper_motion = False

The last step is required to use the `ParametrizedLightCurveMixin`. We need to first load up the light curves that could be used when generating magnitudes for particular visits. This step only needs to be done once when starting up a script to generate catalogs. After this we can generate many catalogs in this notebook without have to load them again.

In [ ]:
ParametrizedLightCurveMixin().load_parametrized_light_curves()

## Start generating catalogs!

First we create an instance of `StarCacheDBObject` to open a connection to our database.

In [ ]:
star_db_name = 'star_cache.db'

In [ ]:
star_db = StarCacheDBObj(star_db_name)

Then we use the `ObservationMetaData` class to generate the pointing information that is similar to what comes out of `OpSim` and defines a visit. Below we generate 30 visits separated by 2 days each.

In [ ]:
obsMetaDataResults = []
day_on = 0
day_0 = 59580.+2720.

for obsHistID in range(30):
    obs = ObservationMetaData(pointingRA=53.00, pointingDec= -24.8,
                              boundType='circle', boundLength=.1, mjd=day_0+day_on)
    obsMetaDataResults.append(obs)
    day_on += 2

Finally, we step through our visit list and generate a new `testCatalogPoint` instance for each catalog we want to write. We then use our database connection and visit metadata to generate the catalog and write it out.

In [ ]:
mjd_list = []
for i in range(30):
    print(i)
    obs_md = obsMetaDataResults[i]
    mjd_list.append(obs_md.mjd.TAI)
    star_cat = testCatalogPoint(star_db, obs_metadata=obsMetaDataResults[i])
    star_cat.write_catalog('star_cat_tvs_%i.txt' % i, write_header=False)

## What's in a catalog?

Below we show some of the contents of what went into our catalogs. Here we have also written out the `varParamStr` information to see what objects there are and what variability models they use.

In [ ]:
!head -10 star_cat_tvs_0.txt

In [ ]:
!tail -10 star_cat_tvs_0.txt

## Using catalogs

Here we load up one of our catalogs in pandas to see what's inside and actually use the data. We load up the object id, PhoSim magNorm value and the lsst r magnitude for the visit.

In [ ]:
cat_0 = pd.read_csv('star_cat_tvs_0.txt', delimiter=' ', usecols=(1,4,5), names=('id', 'magNorm', 'lsstr'))

In [ ]:
cat_0.tail(10)

Looking back at when we printed out the contents of our instance catalog we see that the object 7 lines up from the bottom with object id 854259716 was an RR Lyrae object. Let's step through the catalogs and query for the lsst r magnitude in each visit.

In [ ]:
rrly_curve = []
for i in range(30):
    cat_0 = pd.read_csv('star_cat_tvs_%i.txt' % i, delimiter=' ', usecols=(1,4,5), names=('id', 'magNorm', 'lsstr'))
    rrly_curve.append(cat_0.query('id == 854259716')['lsstr'].values)

Finally, let's plot it and see what we get.

In [ ]:
fig = plt.figure(figsize=(12, 9))
plt.plot(mjd_list, rrly_curve, '--o', lw=4, ms=20)
plt.ylabel('lsst_r', size=18)
plt.xlabel('mjd', size=18)
plt.title('Sample RR Lyrae lightcurve', size=24)

## Loading visits from an Opsim database

First let's download a small testing database from the Twinkles project.

In [ ]:
!wget https://dirac.astro.washington.edu/~brycek/enigma_1189_micro.db

Below we have a set of visits that are in the Twinkles field and we will use the `ObservationMetaDataGenerator` to create a visit list and write catalogs of those visits as before.

In [ ]:
opsimDB = '/home/docmaf/maf_local/enigma_1189_micro.db'
generator = ObservationMetaDataGenerator(database=opsimDB, driver='sqlite')
obsHistIDList = [191578,
 191613,
 193481,
 193512,
 203512,
 210625,
 210655,
 210723,
 211918,
 211938]
obsMetaDataResults = []

The code below uses a visit id and selects a field of view around the center of the visit to generate queries to the database for our catalogs.

In [ ]:
for obsHistID in obsHistIDList:
    obsMetaDataResults.append(generator.getObservationMetaData(obsHistID=obsHistID,
                                  fieldRA=(53, 54),
                                  fieldDec=(-29, -27),
                                  boundLength=0.1)[0])

In [ ]:
mjd_list = []
for i in range(10):
    print(i)
    obs_md = obsMetaDataResults[i]
    mjd_list.append(obs_md.mjd.TAI)
    star_cat = testCatalogPoint(star_db, obs_metadata=obsMetaDataResults[i])
    star_cat.write_catalog('star_cat_tvs_%i.txt' % i, write_header=False)

Once again we can look at the catalog and pick an object to plot. This time let's plot one of the stars with a Kepler based light curve.

In [ ]:
! tail -10 star_cat_tvs_0.txt

In [ ]:
light_curve = []
for i in range(10):
    cat_0 = pd.read_csv('star_cat_tvs_%i.txt' % i, delimiter=' ', usecols=(1,4,5), names=('id', 'magNorm', 'lsstr'))
    light_curve.append(cat_0.query('id == 470323051524')['lsstr'].values)

In [ ]:
fig = plt.figure(figsize=(12, 9))
plt.plot(mjd_list, light_curve, '--+', lw=4, ms=46, markeredgewidth=2)
plt.ylabel('lsst_r', size=18)
plt.xlabel('mjd', size=18)
plt.title('Sample kepler lightcurve', size=24)